In [1]:
from tools.numerical_gradient import *
from models.layers import *
from models.networks.vanilla_rnn import *
import numpy as np

# Let's try out some simple functions for numerical_gradient. #

We know the linear equation y = 3x should always return 3. Let's check it:

In [2]:
def linear(x, slope=3):
    return slope*x

slope = numerical_gradient_check_scalar(linear, 5)
print slope

2.99999999989


In [3]:
def multi_cubic_field(x):
    return np.array([x[0]**3 + x[1]**2, x[0]*2 + x[1]/12])
arr = np.array([3,4], dtype=np.float32)

def matrix_mult(x, b = np.array([[3,5],[2,1]])):
    return x.dot(b)
                
vector_field = numerical_gradient_check_multivar(multi_cubic_field, arr, np.ones_like(arr))
print vector_field

arr = np.array([[3,4],[1,2]], dtype=np.float32)
vector_field = numerical_gradient_check_multivar(matrix_mult, arr, np.ones_like(arr))
print vector_field

[ 29.03938293   8.09431076]
[[ 8.01086426  3.0040741 ]
 [ 8.01086426  3.0040741 ]]


In [6]:
def affine_transform(w, x, b):
    return x.dot(w) + b

x = np.array([[1,2]], float) # 1 x 2
w = np.array([[3,2,1],[1,2,5]], float) # 2 x 3
b = np.array([[1,5,7]], float) # 1 x 3
dummy = affine_transform(w,x,b)

fw = lambda w: affine_transform(w,x,b)
fx = lambda x: affine_transform(w,x,b)
fb = lambda b: affine_transform(w,x,b)

vector_field = numerical_gradient_check_multivar(fw, w, np.ones_like(dummy))
print "fw : ", vector_field
vector_field = numerical_gradient_check_multivar(fx, x, np.ones_like(dummy))
print "fx : ", vector_field
vector_field = numerical_gradient_check_multivar(fb, b, np.ones_like(dummy))
print "fb : ", vector_field

fw :  [[ 0.99999997  0.99999997  0.99999997]
 [ 1.99999995  1.99999995  1.99999995]]
fx :  [[ 5.99999985  7.9999998 ]]
fb :  [[ 0.99999997  0.99999997  0.99999997]]


# word_embedding_forward/backward #

In [6]:
# Looks good to me
ans = np.array([[[3, 4, 7, 1],
                [3, 4, 7, 1],
                [1, 5, 9, 4]],

               [[1, 5, 9, 4],
                [1, 5, 9, 4],
                [1, 5, 9, 4]],

               [[4, 3, 2, 5],
                [3, 4, 7, 1],
                [4, 3, 2, 5]]])

x = np.array([[1,1,0], [0,0,0], [2,1,2]], int)
words = np.array([[1,5,9,4],[3,4,7,1],[4,3,2,5]])
arr = word_embedding_forward(words, x)

assert np.array_equal(ans, arr)
print arr.shape, "\n", arr

(3, 3, 4) 
[[[3 4 7 1]
  [3 4 7 1]
  [1 5 9 4]]

 [[1 5 9 4]
  [1 5 9 4]
  [1 5 9 4]]

 [[4 3 2 5]
  [3 4 7 1]
  [4 3 2 5]]]


In [7]:
dout = np.array([[[1,2,0,1],[3,2,9,1],[1,2,1,1]],
                 [[3,9,2,4],[1,9,9,0],[2,0,1,6]],
                 [[1,0,1,0],[0,1,0,5],[3,0,0,1]]])

arr = word_embedding_backward(dout, words, x)
ans = np.array([[  7.,  20.,  13.,  11.],
               [  4.,   5.,   9.,   7.],
               [  4.,   0.,   1.,   1.]])

assert np.array_equal(ans, arr)
print arr.shape, "\n", arr

(3, 4) 
[[  7.  20.  13.  11.]
 [  4.   5.   9.   7.]
 [  4.   0.   1.   1.]]


# Tanh Vanilla RNN_step Layer #

In [2]:
# Forward
"""
prev_h = (N,H)
x = (N, V)
W_hh = (H,H)
W_xh = (V,H)
b = (H,)
"""
N = 10
D = 3
H = 2
T = 1
V = 5

prev_h = np.random.random((N,H)) # N = 3, H = 5
x = np.random.random((N,V)) # N = 3, V = 4
W_hh = np.random.random((H,H)) # H = 5
W_xh = np.random.random((V,H)) # V = 4, H = 5
b = np.random.random((H,)) # H = 5

res = rnn_step_forward(prev_h, W_hh, x, W_xh, b) # N = 3, H = 5
gradients = np.random.random(res.shape)
print res.shape

(10, 2)


In [3]:
# Backward
fprev_h = lambda prev_h: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fW_hh = lambda W_hh: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fx = lambda x: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fW_xh = lambda W_xh: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fb = lambda b: rnn_step_forward(prev_h, W_hh, x, W_xh, b)

dprev_h_num = numerical_gradient_check_multivar(fprev_h, prev_h, gradients)
dW_hh_num = numerical_gradient_check_multivar(fW_hh, W_hh, gradients)
dx_num = numerical_gradient_check_multivar(fx, x, gradients)
dW_xh_num = numerical_gradient_check_multivar(fW_xh, W_xh, gradients)
db_num = numerical_gradient_check_multivar(fb, b, gradients)

dW_hh, dW_xh, dprev_h, dx, db = rnn_step_backward(prev_h, W_hh, x, W_xh, b, gradients)
print "dprev_h : ", norm_loss(dprev_h, dprev_h_num)
print "dW_hh : ", norm_loss(dW_hh, dW_hh_num)
print "dx : ", norm_loss(dx, dx_num)
print "dW_xh : ", norm_loss(dW_xh, dW_xh_num)
print "db : ", norm_loss(db, db_num)

dprev_h :  1.26478036279e-08
dW_hh :  1.26060034859e-08
dx :  1.26312324765e-08
dW_xh :  1.26228494426e-08
db :  1.25912258873e-08


# Tanh Vanilla RNN Layer #

In [4]:
# Forward
from tools.numerical_gradient import *
from models.layers import *
import numpy as np
"""
h0 = (N,H)
W_hh = (H,H)
x = (N,T,D)
W_xh = (D,H)
b = (H,)
"""
N = 10
D = 3
H = 2
T = 1

h0 = np.random.random((N,H)) * 2
W_hh = np.random.random((H,H)) * 2
x = np.random.random((N,T,D)) * 2
W_xh = np.random.random((D,H)) * 2
b = np.zeros((H,))
x_copy = x.copy()

h = rnn_forward(x, W_xh, W_hh, b, h0)

models/layers.py:161: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if h0 != None: # Supply an h0 state.


In [5]:
# Backward
fx = lambda x: rnn_forward(x, W_xh, W_hh, b, h0)
fW_xh = lambda W_xh: rnn_forward(x, W_xh, W_hh, b, h0)
fW_hh = lambda W_hh: rnn_forward(x, W_xh, W_hh, b, h0)
fb = lambda b: rnn_forward(x, W_xh, W_hh, b, h0)
fh0 = lambda h0: rnn_forward(x, W_xh, W_hh, b, h0)

dx_num = numerical_gradient_check_multivar(fx, x, np.ones_like(h))
dW_xh_num = numerical_gradient_check_multivar(fW_xh, W_xh, np.ones_like(h))
dW_hh_num = numerical_gradient_check_multivar(fW_hh, W_hh, np.ones_like(h))
db_num = numerical_gradient_check_multivar(fb, b, np.ones_like(h))
dh0_num = numerical_gradient_check_multivar(fh0, h0, np.ones_like(h))

dW_hh, dW_xh, dx, db, dh0 = rnn_backward(x, W_xh, W_hh, b, h0, h, np.ones_like(h))
print dW_hh, dW_hh_num
print db, db_num
print "dW_hh : ", norm_loss(dW_hh, dW_hh_num)
print "dW_xh : ", norm_loss(dW_xh, dW_xh_num)
print "dx : ", norm_loss(dx, dx_num)
print "db : ", norm_loss(db, db_num)
print "dh0 : ", norm_loss(dh0, dh0_num)

Current _db is :  [ 0.02009201  0.05359459]
[[ 0.00228181  0.01266638]
 [ 0.02128555  0.02312966]] [[ 0.00228181  0.01266638]
 [ 0.02128555  0.02312966]]
[ 0.02009201  0.05359459] [ 0.02009201  0.05359459]
dW_hh :  1.27021952225e-08
dW_xh :  1.25570247197e-08
dx :  1.25514118422e-08
db :  1.25975381181e-08
dh0 :  1.26652193442e-08


In [7]:
N, D, H = 3, 10, 4

x = np.linspace(-0.4, 0.7, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.2, 0.5, num=N*H).reshape(N, H)
Wx = np.linspace(-0.1, 0.9, num=D*H).reshape(D, H)
Wh = np.linspace(-0.3, 0.7, num=H*H).reshape(H, H)
b = np.linspace(-0.2, 0.4, num=H)
next_h = rnn_step_forward(prev_h, Wh, x, Wx, b)
expected_next_h = np.asarray([
  [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
  [ 0.66854692,  0.79562378,  0.87755553,  0.92795967],
  [ 0.97934501,  0.99144213,  0.99646691,  0.99854353]])

print next_h

[[-0.58172089 -0.50182032 -0.41232771 -0.31410098]
 [ 0.66854692  0.79562378  0.87755553  0.92795967]
 [ 0.97934501  0.99144213  0.99646691  0.99854353]]


# Affine Layer #

In [8]:
# Forward
"""
h = (N,H)
W_hy = (H,D)
b = (D,)
"""
h = np.random.random((3,5)) # N = 3, H = 5
W_hy = np.random.random((5,7)) # H = 5, D = 7
b = np.random.random((7,)) # D = 7

res = affine_forward(h, W_hy, b) # N = 3, D = 7
gradients = np.random.random(res.shape)
print res.shape

(3, 7)


In [9]:
fh = lambda h: affine_forward(h, W_hy, b)
fW_hy = lambda W_hy: affine_forward(h, W_hy, b)
fb = lambda b: affine_forward(h, W_hy, b)

dh_num = numerical_gradient_check_multivar(fh, h,gradients)
dW_hy_num = numerical_gradient_check_multivar(fW_hy, W_hy,gradients)
db_num = numerical_gradient_check_multivar(fb, b,gradients)

dh, dW_hy, db = affine_backward(h, W_hy, b, gradients)

print "dx : ", norm_loss(dh, dh_num)
print "dW_xh : ", norm_loss(dW_hy, dW_hy_num)
print "db : ", norm_loss(db, db_num)

dx :  1.26315460933e-08
dW_xh :  1.26310672038e-08
db :  1.26310626136e-08


# Affine layer for RNN's #

In [4]:
# Forward
"""
h = (N,T,H)
W_hy = (H,D)
b = (D,)
"""
h = np.random.random((3,7,5)) # N = 3, T = 7, H = 5
W_hy = np.random.random((5,7)) # H = 5, D = 7
b = np.random.random((7,)) # D = 7

res = rnn_affine_forward(h, W_hy, b) # N = 3, D = 7
print res.shape

(3, 7, 7)


In [5]:
fh = lambda h: rnn_affine_forward(h, W_hy, b)
fW_hy = lambda W_hy: rnn_affine_forward(h, W_hy, b)
fb = lambda b: rnn_affine_forward(h, W_hy, b)

dh_num = numerical_gradient_check_multivar(fh, h)
dW_hy_num = numerical_gradient_check_multivar(fW_hy, W_hy)
db_num = numerical_gradient_check_multivar(fb, b)

dh, dW_hy, db = rnn_affine_backward(h, W_hy, b, np.ones_like(res))

print "dx : ", norm_loss(dh, dh_num)
print "dW_xh : ", norm_loss(dW_hy, dW_hy_num)
print "db : ", norm_loss(db, db_num)

dx :  1.26309993476e-08
dW_xh :  1.26313237317e-08
db :  1.26310626624e-08


# Softmax Layer - One of the most important functions in Deep Learning #

In [2]:
# Forward and backwards
"""
x = (N,D)
y = (N,)
"""
x = np.random.random((3,4)) # N = 3, D = 4
y = np.random.randint(4, size=3) # D = 4, N = 3

fx = lambda x: softmax(x, y)[0]

loss, dJ = softmax(x, y)
dJ_num = numerical_gradient_check_multivar(fx, x)
print "dJ : ", norm_loss(dJ, dJ_num)

dJ :  1.26099759008e-08


# SVM Layer - The other most important functions in Deep Learning #

In [2]:
# Forward and backwards
"""
x = (N,D)
y = (N,)
"""
x = np.random.random((3,5)) # N = 3, D = 5
y = np.random.randint(5, size=3) # D = 5, N = 3

fx = lambda x: SVM(x, y)[0]

loss, dJ = SVM(x, y)
dJ_num = numerical_gradient_check_multivar(fx, x)
print "dJ : ", norm_loss(dJ, dJ_num)

dJ :  1.26399460897e-08


# Softmax for RNN layer - the important function compatible for RNN's #

In [3]:
# Forward and backwards
"""
x = (N,T,D)
y = (N,)
"""
x = np.random.random((3,5,4)) # N = 3, T = 5, D = 4
y = np.random.randint(4, size=(3,5)) # D = 4, N = 3

fx = lambda x: rnn_softmax(x, y)[0]

loss, dJ = rnn_softmax(x, y)
dJ_num = numerical_gradient_check_multivar(fx, x)
print "dJ : ", norm_loss(dJ, dJ_num)    

dJ :  1.26306575262e-08


# I needed regularization on my net so I dropped out of UCLA #

In [5]:
# Forward and backwards
"""
x = (N,T,D)
"""
x = np.ones((30,50,40), float) # N = 30, T = 50, D = 40
# by def, 30x50x40 = sum of x = 60,000
p = 0.6 # we should see a number around 36,000

# Numerical checking on this kind of stuff is sketch since we have randomness in the function
print np.sum(x)
x,_ = dropout_forward(x, p)
print np.sum(x)
# These should be relatively similar.

60000.0
mask sum :  36128.0
60213.3333333


# The entire RNN forward() #

In [12]:
N, T, D, H = 2, 3, 4, 5

x = np.linspace(-0.1, 0.3, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.3, 0.1, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.4, num=D*H).reshape(D, H)
Wh = np.linspace(-0.4, 0.1, num=H*H).reshape(H, H)
b = np.linspace(-0.7, 0.1, num=H)

h = rnn_forward(x, Wx, Wh, b, h0)
expected_h = np.asarray([
  [
    [-0.42070749, -0.27279261, -0.11074945,  0.05740409,  0.22236251],
    [-0.39525808, -0.22554661, -0.0409454,   0.14649412,  0.32397316],
    [-0.42305111, -0.24223728, -0.04287027,  0.15997045,  0.35014525],
  ],
  [
    [-0.55857474, -0.39065825, -0.19198182,  0.02378408,  0.23735671],
    [-0.27150199, -0.07088804,  0.13562939,  0.33099728,  0.50158768],
    [-0.51014825, -0.30524429, -0.06755202,  0.17806392,  0.40333043]]])
print h


(array([[[-0.42070749, -0.27279261, -0.11074945,  0.05740409,  0.22236251],
        [-0.39525808, -0.22554661, -0.0409454 ,  0.14649412,  0.32397316],
        [-0.42305111, -0.24223728, -0.04287027,  0.15997045,  0.35014525]],

       [[-0.55857474, -0.39065825, -0.19198182,  0.02378408,  0.23735671],
        [-0.27150199, -0.07088804,  0.13562939,  0.33099728,  0.50158768],
        [-0.51014825, -0.30524429, -0.06755202,  0.17806392,  0.40333043]]]), array([[[-0.3       , -0.25555556, -0.21111111, -0.16666667, -0.12222222],
        [-0.42070749, -0.27279261, -0.11074945,  0.05740409,  0.22236251],
        [-0.39525808, -0.22554661, -0.0409454 ,  0.14649412,  0.32397316]],

       [[-0.07777778, -0.03333333,  0.01111111,  0.05555556,  0.1       ],
        [-0.55857474, -0.39065825, -0.19198182,  0.02378408,  0.23735671],
        [-0.27150199, -0.07088804,  0.13562939,  0.33099728,  0.50158768]]]))


# One pass #

In [2]:
N, D, V, H, T = 1,2,2,1,2

model = VanillaRNN(N, D, T, H, V)

np.random.seed(5)
# Set all model parameters to fixed values
for k, v in model.params.iteritems():
    model.params[k] = np.linspace(-1.4, 1.3, num=v.size).reshape(*v.shape)
    print k
    print model.params[k].shape

h0 = np.linspace(-1.5, 0.3, num=(N * H)).reshape(N, H)
captions = (np.arange(N * T) % V).reshape(N, T)
print h0
print captions
x = captions[:,:-1]
y = captions[:,1:]

loss, grads, _ = model.loss(x, y, h0)
print loss

b_rnn
(1,)
W_xh
(2, 1)
W_hy
(1, 2)
words
(2, 2)
b_affine
(2,)
W_hh
(1, 1)
[[-1.5]]
[[0 1]]
words:  [[[-1.4 -0.5]]]
affine_out:  [[-1.5]]
h:  [[[ 0.96472732]]]
t_affine_scores :  [[[-2.75061825  2.55414552]]]
0.00495557253145


models/layers.py:162: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if h0 != None: # Supply an h0 state.
